In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import sqlite3 # library for working with sqlite database
conn = sqlite3.connect("./data/data.db") # Create a connection to the on-disk database

In [2]:
pd.read_sql("SELECT * FROM sqlite_master where type='table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,patients,patients,2,"CREATE TABLE patients(ROW_ID INT,SUBJECT_ID IN..."
1,table,admissions,admissions,639,"CREATE TABLE admissions(ROW_ID INT,SUBJECT_ID ..."
2,table,icustays,icustays,3515,"CREATE TABLE icustays(ROW_ID INT,SUBJECT_ID IN..."
3,table,diagnoses_icd,diagnoses_icd,4944,"CREATE TABLE diagnoses_icd(ROW_ID INT,SUBJECT_..."
4,table,drgcodes,drgcodes,9061,"CREATE TABLE drgcodes(ROW_ID INT,SUBJECT_ID IN..."


In [ ]:
pd.read_sql("""SELECT * FROM admissions LIMIT 10""", conn)


In [ ]:
pat_df = pd.read_sql("SELECT * FROM patients", conn)
adm_df = pd.read_sql("SELECT * FROM admissions", conn)
pat_df.shape[0]

In [ ]:
pat_df.head()

In [ ]:
all_admissions = pd.read_sql("""SELECT * FROM admissions""", conn)
print(all_admissions.shape)
all_icustays = pd.read_sql("""SELECT * from icustays""", conn)
print(all_icustays.shape)
all_icustays.head()

In [3]:
# 'First' admission if it is a patient's first ever admission
# Exclude patients who die during their 'first' visit

one_admission_deaths = pd.read_sql("""SELECT SUBJECT_ID FROM admissions
WHERE HOSPITAL_EXPIRE_FLAG = 1
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) = 1
""", conn)
print("People who died on their first admission:")
print(one_admission_deaths.shape[0])
print(one_admission_deaths.head())

one_admission_survivors = pd.read_sql("""SELECT SUBJECT_ID FROM admissions
WHERE HOSPITAL_EXPIRE_FLAG = 0
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) = 1
ORDER BY CAST(SUBJECT_ID AS UNSIGNED) ASC
""", conn)
print("People who only had one admission but survived it:")
print(one_admission_survivors.shape[0])
print(one_admission_survivors.head())

mult_admissions = pd.read_sql("""SELECT SUBJECT_ID FROM admissions
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) > 1
ORDER BY CAST(SUBJECT_ID AS UNSIGNED) ASC
""", conn)
print("People who had multiple admissions:")
print(mult_admissions.shape[0])
print(mult_admissions.head())

# Concatenate all subject IDs from patients who survived their first visit
patients_cols = pd.concat([one_admission_survivors, mult_admissions], axis = 0)
patients_cols.columns = patients_cols.columns.str.strip()
patients_cols = patients_cols.sort_values('SUBJECT_ID')
print("Patients who survived their first visit: " + str(patients_cols.shape[0]))
patients_cols[5:15]

People who died on their first admission:
5772
   SUBJECT_ID
0           9
1          12
2          21
3          31
4          56
People who only had one admission but survived it:
35522
   SUBJECT_ID
0           2
1           3
2           4
3           5
4           6
People who had multiple admissions:
7537
   SUBJECT_ID
0          17
1          21
2          23
3          34
4          36
Patients who survived their first visit: 43059


,SUBJECT_ID
5,7
6,8
7,10
8,11
9,13
10,16
0,17
11,18
12,19
13,20


In [ ]:
# Get HADM_ID for first admission of each relevant patient


In [4]:
# Features for all patients
# Only take table info for subjects that are included in our design matrix
# Filter via inner join
admissions_info = pd.read_sql("""SELECT adm.SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME
                                 FROM admissions adm
                                 INNER JOIN patients pat
                                 ON adm.SUBJECT_ID = pat.SUBJECT_ID
                                 ORDER BY pat.SUBJECT_ID ASC
                                 """, conn)
admissions_info['DISCHTIME'] = pd.to_datetime(admissions_info['DISCHTIME'])
admissions_info['ADMITTIME'] = pd.to_datetime(admissions_info['ADMITTIME'])

all_icustays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, OUTTIME, LAST_CAREUNIT, LOS
                              FROM icustays icu
                              INNER JOIN patients pat
                              ON icu.SUBJECT_ID = pat.SUBJECT_ID
                              ORDER BY pat.SUBJECT_ID ASC
                              """, conn)

diagnoses_icd_feats = pd.read_sql("""SELECT diag.SUBJECT_ID, HADM_ID, ICD9_CODE
                                  FROM diagnoses_icd diag
                                  INNER JOIN patients pat
                                  ON diag.SUBJECT_ID = pat.SUBJECT_ID
                                  ORDER BY pat.SUBJECT_ID ASC
                                  """, conn)

# Left joined DRG codes because a lot of DRG severity/mortality data is missing
drgcodes_feats = pd.read_sql("""SELECT drg.SUBJECT_ID, HADM_ID, DRG_SEVERITY, DRG_MORTALITY 
                             FROM drgcodes drg
                             LEFT JOIN patients pat
                             ON drg.SUBJECT_ID = pat.SUBJECT_ID
                             ORDER BY pat.SUBJECT_ID ASC
                             """, conn)

In [5]:
# Further process information for admissions and ICU stays
# Calculate length of stay for each admission
admissions_feats= pd.read_sql("""SELECT adm.SUBJECT_ID, HADM_ID
                                 FROM admissions adm
                                 INNER JOIN patients pat
                                 ON adm.SUBJECT_ID = pat.SUBJECT_ID
                                 ORDER BY pat.SUBJECT_ID ASC
                                 """, conn)
# 'HOS_LOS' is length of stay in HOSPITAL, in seconds ('TimedeltaProperties' object only
# has days, seconds, microseconds as attributes)
admissions_feats['HOS_LOS'] = (admissions_info['DISCHTIME'] - admissions_info['ADMITTIME']).dt.seconds

# Only get information from LAST ICU stay from an admission
# icu_stay_counts = all_icustays['HADM_ID'].value_counts()
single_icu_stays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, LAST_CAREUNIT, LOS
                                    FROM icustays icu
                                    INNER JOIN patients pat
                                    ON icu.SUBJECT_ID = pat.SUBJECT_ID
                                    GROUP BY HADM_ID
                                    HAVING COUNT(HADM_ID) = 1
                                    ORDER BY icu.SUBJECT_ID ASC
                                    """, conn)
multiple_icu_stays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, LAST_CAREUNIT, LOS
                                    FROM icustays icu
                                    INNER JOIN patients pat
                                    ON icu.SUBJECT_ID = pat.SUBJECT_ID
                                    GROUP BY HADM_ID
                                    HAVING COUNT(HADM_ID) > 1
                                    ORDER BY icu.SUBJECT_ID ASC
                                    """, conn)
print(single_icu_stays.shape)
print(multiple_icu_stays.shape)
multiple_icu_stays.columns = multiple_icu_stays.columns.str.strip()

(54526, 4)
(3260, 4)


In [33]:
all_icustays[40:45]

,SUBJECT_ID,HADM_ID,OUTTIME,LAST_CAREUNIT,LOS
40,38,185910,2166-09-04 13:39:23,TSICU,25.5485
41,39,106266,2114-12-09 15:10:00,NICU,9.7451
42,41,101757,2133-01-06 16:01:33,SICU,3.3937
43,41,101757,2133-01-12 15:51:03,CSRU,3.1476
44,42,119203,2116-04-28 16:39:05,CCU,1.9024


In [31]:
all_icustays_sorted = all_icustays.sort_values('OUTTIME', ascending=False)
icustays_feats = all_icustays_sorted.drop_duplicates(subset=['HADM_ID'], keep='first')
icustays_feats = icustays_feats.sort_values('SUBJECT_ID')
icustays_feats[4:16]

,SUBJECT_ID,HADM_ID,OUTTIME,LAST_CAREUNIT,LOS
4,6,107064,2175-06-03 13:39:54,SICU,3.6729
6,7,118037,2121-05-25 21:10:19,NICU,0.7391
7,8,159514,2117-11-21 14:24:55,NICU,1.0755
8,9,150750,2149-11-14 20:52:14,MICU,5.3231
9,10,184167,2103-07-06 13:51:43,NICU,8.0921
10,11,194540,2178-04-17 20:21:05,SICU,1.5844
11,12,112213,2104-08-15 17:22:25,SICU,7.6348
12,13,143045,2167-01-12 10:43:31,CSRU,3.666
13,16,103251,2178-02-03 08:59:00,NICU,0.0979
15,17,161087,2135-05-10 11:18:34,CSRU,0.8795


In [ ]:
# Age
pat_adm_df = pd.read_sql("""SELECT pat.SUBJECT_ID, pat.DOB, adm.ADMITTIME
                            FROM patients pat
                            INNER JOIN admissions adm
                            ON pat.SUBJECT_ID = adm.SUBJECT_ID
                            ORDER BY pat.SUBJECT_ID ASC""", conn)

admissions_feats['age'] = (pd.to_datetime(pat_adm_df['ADMITTIME']) - pd.to_datetime(pat_adm_df['DOB'])).dt.days

In [ ]:
print(admissions_info.shape)
admissions_info.head()

In [ ]:
admissions_feats.head()

In [ ]:
print(all_icustays.shape)
all_icustays[0:10]

In [ ]:
print(diagnoses_icd_feats.shape)
diagnoses_icd_feats.head()

In [ ]:
print(drgcodes_feats.shape)
drgcodes_feats[15:25]

In [ ]:
# Get one overall features matrix for ALL patients
features_for_all_mat = pd.merge(admissions_feats, icustays_feats[['HADM_ID', 'LAST_CAREUNIT', 'LOS']], how='left', on=['HADM_ID'])
print(features_for_all_mat.shape)

# Merge feature dataframe with the patient_cols dataframes so we only keep the features
# for the RELEVANT patients
features_mat = pd.merge(patients_cols, features_for_all_mat, how='left', on=['SUBJECT_ID'])
print(features_mat.shape)
features_mat[0:20]

In [ ]:
# Generate additional columns:
# Append truth labels column (Was patient readmitted within 30 days?)
# Append 'ICU stay?' column
subj_adm = admissions_feats["SUBJECT_ID"].value_counts()
subj_adm = subj_adm.sort_index()

mult_adm_subj = subj_adm.index[subj_adm > 1]

# Instantiate to all zeros
truth = np.zeros((len(subj_adm)))

subj_adm_idx = list(subj_adm.index)

for subject in mult_adm_subj:
    temp_df = admissions_info[admissions_info["SUBJECT_ID"] == subject]
    # Get time between first discharge and second admission
    first_disc_time = temp_df["DISCHTIME"].iloc[0]
    second_adm_time = temp_df["ADMITTIME"].iloc[1]
    days_to_readm = (pd.to_datetime(second_adm_time) - pd.to_datetime(first_disc_time)).days
    
    # If readmitted within 30 days, set truth to 1
    if days_to_readm <= 30:
        truth[subj_adm_idx.index(subject)] = 1
        
truth_df = pd.DataFrame({'SUBJECT_ID': subj_adm_idx, 'READM': np.array(truth, dtype='int')})
print(truth_df.head())

In [ ]:
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn import metrics
from sklearn.model_selection import train_test_split

design_mat = features_mat.merge(truth_df,how='inner',on="SUBJECT_ID")
X = design_mat.iloc[:,2:-1]

LCU = design_mat["LAST_CAREUNIT"]
le = LabelEncoder() # categorical encoder
LCU = le.fit_transform(LCU.astype(str))
X['LAST_CAREUNIT'] = LCU
X['LOS'] = pd.to_numeric(X['LOS']).fillna(0)

# normalize feature values
for i in list(X):
    X[i] = normalize(X[i][:].values.reshape(-1,1),axis=0)
    
# make train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, design_mat.iloc[:,-1],test_size = 0.33, random_state = 0)

X

### Classifiers

### Metrics for evaluating models

In [ ]:
# provide these metrics
# y_pred = # this is the output prediction values for the classifier
# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
# AUC = metrics.auc(fpr, tpr)
# AUC